In [ ]:
from langchain_yt_dlp import YoutubeLoaderDL
from langchain_community.document_loaders import YoutubeLoader
from langchain_community.document_loaders.youtube import TranscriptFormat
from yt_dlp import YoutubeDL

# Load the video metadata and transcript
def load_youtube_video(url):
    metadata_loader = YoutubeLoaderDL.from_youtube_url(
        url,
        add_video_info=True,
    )

    transcript_loader = YoutubeLoader.from_youtube_url(
        url,
        add_video_info=False,
        transcript_format=TranscriptFormat.CHUNKS
    )

    metadata_docs = metadata_loader.load()
    transcript_docs = transcript_loader.load()

    for i, doc in enumerate(transcript_docs):
        doc.metadata.update(metadata_docs[0].metadata)
        doc.metadata['source_type'] = 'youtube'
        doc.metadata['chunk_number'] = i + 1
    
    return transcript_docs

# Load all videos from a YouTube channel
def load_youtube_channel(channel_url):
    ydl_opts = {
        'extract_flat': True,
        'quiet': True,
        'force_generic_extractor': True,
    }

    print(f"Loading channel: {channel_url}")

    with YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(channel_url, download=False)
    
    print(f"Found {len(info.get('entries', []))} videos in channel: {info.get('title', 'Unknown Channel')}")
    
    if 'entries' not in info:
        raise ValueError("No entries found in the channel URL.")
    
    documents = []
    for entry in info['entries']:
        video_url = entry['url']
        print(f"Loading video: {entry['title']} ({video_url})")
        docs = load_youtube_video(video_url)
        documents.extend(docs)
    
    return documents

# documents = load_youtube_channel('https://www.youtube.com/@UVAResearchComputing')
documents = load_youtube_channel('https://www.youtube.com/playlist?list=PLT4bryHgBcRP7N-hB9u6EWs6tq_2nMoRO')

In [2]:
# print unique video metadatas
import json

unique_metadata = {json.dumps({k: v for k, v in d.metadata.items() if k not in ['chunk_number', 'start_seconds', 'start_timestamp']}, sort_keys=True) for d in documents}

print(f"Total unique video metadata entries: {len(unique_metadata)}")

for metadata in unique_metadata:
    print(json.loads(metadata))

NameError: name 'documents' is not defined

In [3]:
from chromadb import PersistentClient
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

CHROMA_PATH = "./chroma"
COLLECTION_NAME = "data"

embeddings = OllamaEmbeddings(
    base_url="http://localhost:11434",
    model="qwen3"
)

persistent_client = PersistentClient(
    path=CHROMA_PATH,
)

collection = persistent_client.get_or_create_collection(name=COLLECTION_NAME)

vector_store = Chroma(
    client=persistent_client,
    collection_name=COLLECTION_NAME,
    embedding_function=embeddings,
)

# Add documents to the vector store
def add_documents_to_vector_store(documents):
    vector_store.add_documents(
        documents=documents,
        ids=[doc.metadata['source_type'] + "_" + doc.metadata['source'] + "_" + str(doc.metadata['chunk_number']) for doc in documents],
    )
    print(f"Added {len(documents)} documents to the vector store.")

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [ ]:
add_documents_to_vector_store(documents)